In [ ]:
! pip install git+https://github.com/openai/whisper.git

In [ ]:
# import libraries
import os
import numpy as np
import pandas as pd
import whisper

from tqdm import tqdm

In [ ]:
model = whisper.load_model("base")

In [ ]:
# get audio clips and transcriptions
import os

recs = {}

for path, _, files in os.walk('./ravdess-song/'):
    for file in files:
        if file[12:14] == "01": # parse file name to get sentence
            txt = "Kids are talking by the door"
        else:
            txt = "Dogs are sitting by the door"
        
        recs[os.path.join(path, file).replace('\\', '/')] = txt

recs

In [ ]:
# choose 100 random clips
import random

clips = []
for i in range(100):
    clips.append(random.choice(list(recs.items())))

clips

In [ ]:
# get results
results = []
for clip in tqdm(clips):
    result = model.transcribe(clip[0], **dict(without_timestamps=True))["text"]
    results.append(result)

results

In [ ]:
from whisper.normalizers import EnglishTextNormalizer

normalizer = EnglishTextNormalizer()

data = pd.DataFrame(dict(transcription=results, reference=[clip[1] for clip in clips]))
data["transcription"] = [normalizer(text) for text in data["transcription"]]
data["reference"] = [normalizer(text) for text in data["reference"]]
data

In [ ]:
! pip install jiwer

In [ ]:
import jiwer

wer = jiwer.wer(list(data["reference"]), list(data["transcription"]))

print(f"WER: {wer * 100:.2f} %")